In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
import SimpleITK as sitk

# Пути к двум сравниваемым .nii.gz файлам
file1_path = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid/valid_1/valid_1_a/valid_1_a_1.nii.gz'
file2_path = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed/valid_1/valid_1_a/valid_1_a_1.nii.gz'

# Загрузка данных из nii.gz файлов
img1 = sitk.ReadImage(file1_path)
img2 = sitk.ReadImage(file2_path)

metadata_path = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/metadata/validation_metadata.csv'
metadata = pd.read_csv(metadata_path)

In [2]:
row = metadata[metadata['VolumeName'] == file1_path.split('/')[-1]]
row

,VolumeName,Manufacturer,SeriesDescription,ManufacturerModelName,PatientSex,PatientAge,ReconstructionDiameter,DistanceSourceToDetector,DistanceSourceToPatient,GantryDetectorTilt,...,PhotometricInterpretation,Rows,Columns,XYSpacing,RescaleIntercept,RescaleSlope,RescaleType,NumberofSlices,ZSpacing,StudyDate
0,valid_1_a_1.nii.gz,Philips,HRCT,iCT 256,M,036Y,350.0,NaN,NaN,0,...,MONOCHROME2,1024,1024,"[0.341796875, 0.341796875]",-1024,1,NaN,251,1.5,20210427


In [3]:
import ast

# Set Spacing
(x, y), z = map(float, ast.literal_eval(row['XYSpacing'].iloc[0])), row['ZSpacing'].iloc[0]
img1.SetSpacing((x,y,z))

# Set Origin
img1.SetOrigin(ast.literal_eval(row['ImagePositionPatient'].iloc[0]))

# Set Direction
orientation = ast.literal_eval(row['ImageOrientationPatient'].iloc[0])
row_cosine, col_cosine = orientation[:3], orientation[3:6]
z_cosine = np.cross(row_cosine, col_cosine).tolist()
img1.SetDirection(row_cosine + col_cosine + z_cosine)

# Fix Rescale
RescaleIntercept = row['RescaleIntercept'].iloc[0]
RescaleSlope = row['RescaleSlope'].iloc[0]
adjusted_hu = img1 * RescaleSlope + RescaleIntercept

# Convert the image to int16
adjusted_hu = sitk.Cast(adjusted_hu, sitk.sitkInt16)


In [4]:
np.allclose(sitk.GetArrayFromImage(adjusted_hu), sitk.GetArrayFromImage(img2))

True